In [1]:
import pyspark as sp
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,date_format,concat,substring,lit,lpad,to_timestamp,when,mean,avg,reduce
import  pandas as pd
import os

In [2]:
spark = SparkSession.builder \
    .appName("PreProcessing") \
    .getOrCreate()

24/04/21 20:48:55 WARN Utils: Your hostname, Subins-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.65.210.91 instead (on interface en0)
24/04/21 20:48:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/21 20:48:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
for filename in os.listdir("./dataPreprocessed"):
    #   entry=os.path.join("./dataConverted",dir)  
    #   if os.path.isdir(entry):
            #for filename in os.listdir(entry):
                  df=spark.read.option("header","true").option("inferSchema", "true").csv("./dataPreprocessed/"+filename)
                  df = df.withColumn("month", substring(df["date"], 6, 2))
                  avg_values_month = df.groupBy('month').agg(
    avg('ele').alias('avg_ele'),
    avg('winddir').alias('avg_winddir'),
    avg('sky').alias('avg_sky'),
    avg('visdist').alias('avg_visdist'),
    avg('airtemp').alias('avg_airtemp'),
    avg('dewpointtemp').alias('avg_dewpointtemp'),
    avg('atmpress').alias('avg_atmpress')
)

                  avg_values_month = avg_values_month.orderBy("month")
                  pd_df=avg_values_month.toPandas()
                  #df.coalesce(1).write.csv("./dataPreprocessed/"+dir+'/'+dir+'.csv',mode="overwrite", header=True)
                  pd_df.to_csv('./dataPreprocessed/'+filename, index=False,mode='w')


In [6]:
dfs = []

In [7]:
for csv_file in os.listdir("./dataPreprocessed"):

    df=spark.read.option("header","true").option("inferSchema", "true").csv("./dataPreprocessed/"+filename)

   
    dfs.append(df)

merged_df = reduce(lambda df1, df2: df1.union(df2), dfs)

# Extract year from file name and add as a new column
merged_df = merged_df.withColumn("year", csv_file.split(".")[0].cast("int"))

# Sort the DataFrame by "month" and "year"
merged_df = merged_df.orderBy("month", "year")

# Write the merged DataFrame to a new CSV file
merged_df.write.csv("merged_data.csv", header=True)

# Write the merged DataFrame to a new CSV file
merged_df.write.csv("./merged_data.csv", header=True)

TypeError: reduce() missing 1 required positional argument: 'merge'